In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# Files
# =========================
serial_file = "gwo_serial.csv"
openmp_file = "gwo_openmp.csv"
opencl_file = "gwo_opencl.csv"
cuda_file   = "gwo_cuda.csv"

out_dir = Path("plots_ndata")
out_dir.mkdir(exist_ok=True)

# =========================
# Load
# =========================
serial = pd.read_csv(serial_file)
openmp = pd.read_csv(openmp_file)
opencl = pd.read_csv(opencl_file)
cuda   = pd.read_csv(cuda_file)

# =========================
# Column normalization helpers
# =========================
def find_best_sse_col(df: pd.DataFrame) -> str:
    cols = list(df.columns)
    for c in ["best_sse", "best_SSE", "bestSSE", "best_sse_openmp_style", "best_sse_openmp"]:
        if c in cols:
            return c
    low = {c.lower(): c for c in cols}
    for lc, orig in low.items():
        if ("best" in lc) and ("sse" in lc):
            return orig
    raise KeyError(
        "Cannot find best SSE column. "
        "Please ensure CSV has a column like 'best_sse'. "
        f"Available columns: {cols}"
    )

serial_sse_col = find_best_sse_col(serial)
openmp_sse_col = find_best_sse_col(openmp)
opencl_sse_col = find_best_sse_col(opencl)
cuda_sse_col   = find_best_sse_col(cuda)

# Types
for df in [serial, openmp, opencl, cuda]:
    df["Ndata"] = df["Ndata"].astype(int)
    df["POP_SIZE"] = df["POP_SIZE"].astype(int)
    df["avg_ms"] = df["avg_ms"].astype(float)

openmp["threads"] = openmp["threads"].astype(int)

# SSE numeric
serial["best_sse"] = pd.to_numeric(serial[serial_sse_col], errors="coerce")
openmp["best_sse"] = pd.to_numeric(openmp[openmp_sse_col], errors="coerce")
opencl["best_sse"] = pd.to_numeric(opencl[opencl_sse_col], errors="coerce")
cuda["best_sse"]   = pd.to_numeric(cuda[cuda_sse_col], errors="coerce")

# Convenience: seconds
for df in [serial, openmp, opencl, cuda]:
    df["avg_s"] = df["avg_ms"] / 1000.0

# =========================
# Baseline map: (Ndata, POP_SIZE) -> serial avg_ms
# =========================
baseline = (
    serial[["Ndata","POP_SIZE","avg_ms"]]
    .rename(columns={"avg_ms":"T_serial_ms"})
    .copy()
)
baseline = baseline.groupby(["Ndata","POP_SIZE"], as_index=False)["T_serial_ms"].min()

def attach_baseline(df: pd.DataFrame) -> pd.DataFrame:
    out = df.merge(baseline, on=["Ndata","POP_SIZE"], how="left")
    if out["T_serial_ms"].isna().any():
        miss = out[out["T_serial_ms"].isna()][["Ndata","POP_SIZE"]].drop_duplicates()
        raise ValueError("Missing serial baseline for:\n" + miss.to_string(index=False))
    out["speedup"] = out["T_serial_ms"] / out["avg_ms"]
    return out

def karp_flatt(speedup, p):
    s = np.asarray(speedup, dtype=float)
    p = np.asarray(p, dtype=float)
    e = (1/s - 1/p) / (1 - 1/p)
    e[p <= 1] = np.nan
    return e

# =========================
# Metrics
# =========================
openmp_m = attach_baseline(openmp)
openmp_m["efficiency"] = openmp_m["speedup"] / openmp_m["threads"]
openmp_m["karp_flatt_e"] = karp_flatt(openmp_m["speedup"], openmp_m["threads"])

opencl_m = attach_baseline(opencl)
cuda_m   = attach_baseline(cuda)

openmp_m.to_csv("summary_openmp_metrics_ndata.csv", index=False)

# =========================
# Helper: SSE format
# =========================
def fmt_sse(sse):
    if pd.isna(sse):
        return "SSE=NA"
    if abs(sse - round(sse)) < 1e-6:
        return f"SSE={int(round(sse))}"
    return f"SSE={sse:.1f}"

def fmt_sec(ms):
    return f"{ms/1000.0:.3f}s"

# =========================
# 1) SERIAL stacked % per (Ndata) separately
#    REQUIREMENT: add SSE into serial_stack + padding like best_framework_speedup
# =========================
need_cols = [
    "avg_update_pop_excl_ms",
    "avg_update_fitness_excl_ms",
    "avg_fitness_batch_excl_ms",
    "avg_fitness_scalar_ms",
]
for c in need_cols:
    if c not in serial.columns:
        raise KeyError(f"serial missing column: {c}")

ser = serial.copy()
ser["t_update_pop_excl"] = ser["avg_update_pop_excl_ms"].astype(float)
ser["t_update_fitness_excl"] = ser["avg_update_fitness_excl_ms"].astype(float)
ser["t_fitness_batch_excl"] = ser["avg_fitness_batch_excl_ms"].astype(float)
ser["t_fitness_scalar"] = ser["avg_fitness_scalar_ms"].astype(float)

ser["t_accounted"] = (
    ser["t_update_pop_excl"] +
    ser["t_update_fitness_excl"] +
    ser["t_fitness_batch_excl"] +
    ser["t_fitness_scalar"]
)
ser["t_other"] = (ser["avg_ms"] - ser["t_accounted"]).clip(lower=0)

for c in ["t_update_pop_excl","t_update_fitness_excl","t_fitness_batch_excl","t_fitness_scalar","t_other"]:
    ser[c + "_pct"] = 100.0 * ser[c] / ser["avg_ms"]

for ndata in sorted(ser["Ndata"].unique()):
    sub = ser[ser["Ndata"] == ndata].sort_values("POP_SIZE")
    x = np.arange(len(sub))
    labels = [f"POP={p}" for p in sub["POP_SIZE"].tolist()]

    fig, ax = plt.subplots(figsize=(12, 5))
    ax.set_title(f"Serial: % thời gian (exclusive) theo POP_SIZE | Ndata={ndata}")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)

    bottom = np.zeros(len(sub))
    ax.bar(x, sub["t_fitness_scalar_pct"], bottom=bottom, label="fitness_scalar (SSE core)")
    bottom += sub["t_fitness_scalar_pct"].to_numpy()

    ax.bar(x, sub["t_fitness_batch_excl_pct"], bottom=bottom, label="fitness_batch excl (overhead)")
    bottom += sub["t_fitness_batch_excl_pct"].to_numpy()

    ax.bar(x, sub["t_update_fitness_excl_pct"], bottom=bottom, label="update_fitness excl (heap/copy)")
    bottom += sub["t_update_fitness_excl_pct"].to_numpy()

    ax.bar(x, sub["t_update_pop_excl_pct"], bottom=bottom, label="update_pop excl (position update)")
    bottom += sub["t_update_pop_excl_pct"].to_numpy()

    ax.bar(x, sub["t_other_pct"], bottom=bottom, label="other / residue")

    ax.set_ylabel("Percent (%)")
    # padding/headroom giống best_framework_speedup (để nhét label SSE)
    ax.set_ylim(0, 110)
    ax.grid(axis="y", linestyle="--", alpha=0.3)
    ax.legend()
    fig.tight_layout()

    # Annotate SSE above 100% line, with offset + clip_off
    for xi, sse in zip(x, sub["best_sse"].tolist()):
        ax.annotate(
            fmt_sse(sse),
            xy=(xi, 100),
            xytext=(0, 6),
            textcoords="offset points",
            ha="center", va="bottom",
            fontsize=9,
            clip_on=False
        )

    fig.savefig(out_dir / f"serial_stacked_percent_ndata{ndata}.png", dpi=200)
    plt.close(fig)

# =========================
# 2) OpenMP plots per (Ndata, POP)
# =========================
for ndata in sorted(openmp_m["Ndata"].unique()):
    subN = openmp_m[openmp_m["Ndata"] == ndata]
    for pop in sorted(subN["POP_SIZE"].unique()):
        sub = subN[subN["POP_SIZE"] == pop].sort_values("threads")
        if sub.empty:
            continue

        plt.figure()
        plt.plot(sub["threads"], sub["speedup"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Speedup (vs Serial)")
        plt.title(f"OpenMP Speedup | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_speedup_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

        plt.figure()
        plt.plot(sub["threads"], sub["efficiency"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Efficiency")
        plt.title(f"OpenMP Efficiency | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_efficiency_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

        plt.figure()
        plt.plot(sub["threads"], sub["karp_flatt_e"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Karp–Flatt e")
        plt.title(f"OpenMP Karp–Flatt e | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_karpflatt_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

# =========================
# 3) Best framework compare per (Ndata, POP)
#    REQUIREMENT: add runtime into best_framework_speedup (seconds)
# =========================
best_openmp = openmp_m.loc[openmp_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_openmp["framework"] = "OpenMP"
best_openmp["label"] = best_openmp.apply(lambda r: f"OpenMP({int(r['threads'])})", axis=1)

best_opencl = opencl_m.loc[opencl_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_opencl["framework"] = "OpenCL"
# opencl csv uses column 'device' (per your header)
best_opencl["label"] = best_opencl.apply(lambda r: f"OpenCL({r.get('device','dev')})", axis=1)

best_cuda = cuda_m.loc[cuda_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_cuda["framework"] = "CUDA"
# cuda csv uses column 'device' (per your header)
best_cuda["label"] = best_cuda.apply(lambda r: f"CUDA({r.get('device','dev')})", axis=1)

best_compare = pd.concat([
    best_openmp[["Ndata","POP_SIZE","framework","avg_ms","avg_s","speedup","label","best_sse"]],
    best_opencl[["Ndata","POP_SIZE","framework","avg_ms","avg_s","speedup","label","best_sse"]],
    best_cuda[["Ndata","POP_SIZE","framework","avg_ms","avg_s","speedup","label","best_sse"]],
], ignore_index=True)

best_compare.to_csv("summary_best_framework_compare_ndata.csv", index=False)

for ndata in sorted(best_compare["Ndata"].unique()):
    subN = best_compare[best_compare["Ndata"] == ndata]
    for pop in sorted(subN["POP_SIZE"].unique()):
        sub = subN[subN["POP_SIZE"] == pop].sort_values("speedup", ascending=False)
        if sub.empty:
            continue

        fig, ax = plt.subplots(figsize=(11, 4.8))
        bars = ax.bar(sub["label"], sub["speedup"])
        ax.set_xticklabels(sub["label"], rotation=20, ha="right")

        ax.set_ylabel("Speedup vs Serial")
        ax.set_title(f"Best Speedup by Framework | Ndata={ndata}, POP={pop}")
        ax.grid(axis="y", linestyle="--", alpha=0.3)

        ymax = float(sub["speedup"].max())
        ax.set_ylim(0, ymax * 1.25 if ymax > 0 else 1.0)

        # REQUIREMENT: annotate both SSE + time (seconds)
        for b, sse, ms in zip(bars, sub["best_sse"].tolist(), sub["avg_ms"].tolist()):
            txt = f"{fmt_sse(sse)}\nT={fmt_sec(ms)}"
            ax.annotate(
                txt,
                xy=(b.get_x() + b.get_width()/2.0, b.get_height()),
                xytext=(0, 6),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=9,
                clip_on=False
            )

        fig.tight_layout()
        fig.savefig(out_dir / f"best_framework_speedup_ndata{ndata}_pop{pop}.png", dpi=170)
        plt.close(fig)

# =========================
# 4) OpenCL device compare by SPEEDUP (vs Serial)
#    include SSE + time (seconds)
# =========================
# Pick best (min avg_ms) per (Ndata, POP, device) to avoid multiple configs per device
if "device" not in opencl_m.columns:
    opencl_m["device"] = "OpenCL-Device"

opencl_best_by_dev = opencl_m.loc[
    opencl_m.groupby(["Ndata", "POP_SIZE", "device"])["avg_ms"].idxmin()
].copy()

for ndata in sorted(opencl_best_by_dev["Ndata"].unique()):
    subN = opencl_best_by_dev[opencl_best_by_dev["Ndata"] == ndata]
    for pop in sorted(subN["POP_SIZE"].unique()):
        sub = subN[subN["POP_SIZE"] == pop].sort_values("speedup", ascending=False)
        if sub.empty:
            continue

        labels = sub["device"].astype(str).tolist()

        fig, ax = plt.subplots(figsize=(12, 4.8))
        bars = ax.bar(labels, sub["speedup"])

        ax.set_ylabel("Speedup vs Serial")
        ax.set_title(f"OpenCL Device Speedup Compare | Ndata={ndata}, POP={pop}")
        ax.set_xticklabels(labels, rotation=20, ha="right")
        ax.grid(axis="y", linestyle="--", alpha=0.3)

        ymax = float(sub["speedup"].max())
        ax.set_ylim(0, ymax * 1.25 if ymax > 0 else 1.0)

        # annotate SSE + runtime seconds
        for b, sse, ms in zip(bars, sub["best_sse"].tolist(), sub["avg_ms"].tolist()):
            txt = f"{fmt_sse(sse)}\nT={ms/1000.0:.3f}s"
            ax.annotate(
                txt,
                xy=(b.get_x() + b.get_width()/2.0, b.get_height()),
                xytext=(0, 6),
                textcoords="offset points",
                ha="center", va="bottom",
                fontsize=9,
                clip_on=False
            )

        fig.tight_layout()
        fig.savefig(out_dir / f"opencl_device_speedup_ndata{ndata}_pop{pop}.png", dpi=170)
        plt.close(fig)

print("Saved:")
print(" - summary_openmp_metrics_ndata.csv")
print(" - summary_best_framework_compare_ndata.csv")
print(f" - plots in: {out_dir.resolve()}")


C:\Users\nguye\AppData\Local\Temp\ipykernel_10544\2081547360.py:272: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(sub["label"], rotation=20, ha="right")
C:\Users\nguye\AppData\Local\Temp\ipykernel_10544\2081547360.py:272: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(sub["label"], rotation=20, ha="right")
C:\Users\nguye\AppData\Local\Temp\ipykernel_10544\2081547360.py:272: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(sub["label"], rotation=20, ha="right")
C:\Users\nguye\AppData\Local\Temp\ipykernel_10544\2081547360.py:272: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(sub["label"

Saved:
 - summary_openmp_metrics_ndata.csv
 - summary_best_framework_compare_ndata.csv
 - plots in: D:\C_C++\kmean_clustering\plots_ndata
